# Summary
This notebook is used to automatically extract data from ATB yearly workbook file and generate summary accordingly to the specification.

## Getting Ready
- Here, we first import the package needed, and identify the path to files;
    - 2 paths to files are required here:
        1. raw_data_path: the path to the raw data file
        2. ancillary_path: the path to the ancillary file which declares the scope of tech and index of interest.

- Make 2 lists out of the ancillary file

In [39]:
import pandas as pd

In [40]:
raw_data_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/2022 v2 Annual Technology Baseline Workbook Corrected 7-21-2022.xlsx'

ancillary_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/ancillary.xlsx'

raw = pd.ExcelFile(raw_data_path)

summary_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/summary.xlsx'

del raw_data_path

In [41]:
ancillary = pd.read_excel(ancillary_path)

# tech: sheet dictionary
tech_sheet_dict = dict(ancillary[['Tech', 'Sheet']].values)

# make tech list out of ancillary
tech_list = list(ancillary['Tech'].dropna().values)

# make index list out of ancillary
index_list = list(ancillary['Index'].dropna().values)

# make year list out of ancillary
year_list = list(ancillary['Year'].dropna().values)

# make translation dictionary out of ancillary
tech_chinese = dict(ancillary[['Tech', 'Chinese']].values)

# flush-flush
del ancillary, ancillary_path

## Main loop
- Iterate over tech sheets needed, and get rid of the extraneous rows.
- Make a dictionary of each sheet to store their corresponding dataframe

In [42]:
# get all the name of the sheets -> intersect with tech list
sheet_list = list(set(raw.sheet_names) & set(tech_list))
sub_tech_list = list(set(tech_list)-set(sheet_list))

In [43]:
sheets = {}
last_i = None

# iterate through all sheets level techs
for sheet in sheet_list:
    # special specification for storages
    if not (sheet.split()[-1] == 'Storage'):
        sheet_df = pd.read_excel(raw, sheet_name=str(sheet)).iloc[:, 9:].dropna(how='all')
    else:
        sheet_df = pd.read_excel(raw, sheet_name=str(sheet)).iloc[:, 3:].dropna(how='all')
    index = None
    # iterate over rows
    for i in range(len(sheet_df)):
        # get the value on the current index column
        working_index = sheet_df.iloc[i, 0]

        if not pd.isna(working_index):
            sheet_df.iloc[i-1, 0] = None    # get rid of the header (year) row
            if working_index in index_list:
                index = working_index
                last_i = i
            else:
                index = None

        sheet_df.iloc[i, 0] = index

    # get the header
    header = list(sheet_df.iloc[last_i-1, :].values)
    header[0:3] = ['Index', 'Display Name', 'Scenario']
    sheet_df.columns = header

    sheet_df = sheet_df.dropna(how='any', subset=['Index']).reset_index(drop=True)
    sheets[str(sheet)] = sheet_df

- for those tech that is relatively more detailed, we shall first generate a dictionary of its correspondence to the parent sheet it belongs to.
- For now, we are trying to match the sub-techs with the sheet which has a name that is most similar to it.
- If it can't work, we will switch to some pristine methods for example manual specification. **which is likely the case** (modification on the ancillary file)

In [ ]:
# iterate over the sub-tech groups
for subtech in sub_tech_list:
    # visit the sheet which contains the tech
    sheet_name = tech_sheet_dict[subtech]
    # read the sheet
    if not (sheet_name.split()[-1] == 'Storage'):
        sheet_df = pd.read_excel(raw, sheet_name=str(sheet_name)).iloc[:, 9:].dropna(how='all')
    else:
        sheet_df = pd.read_excel(raw, sheet_name=str(sheet_name)).iloc[:, 3:].dropna(how='all')

    index = None
    # iterate over rows
    for i in range(len(sheet_df)):
        # get the value on the current index column
        working_index = sheet_df.iloc[i, 0]

        if not pd.isna(working_index):
            sheet_df.iloc[i-1, 0] = None    # get rid of the header (year) row
            if working_index in index_list:
                index = working_index
                last_i = i
            else:
                index = None

        sheet_df.iloc[i, 0] = index

    # get the header
    header = list(sheet_df.iloc[last_i-1, :].values)
    header[0:3] = ['Index', 'Display Name', 'Scenario']
    sheet_df.columns = header

    sheet_df = sheet_df.dropna(how='any', subset=['Index']).reset_index(drop=True)

    # filter the dataframe
    sheet_df = sheet_df[sheet_df['Display Name']==str(subtech)]
    sheets[str(subtech)] = sheet_df

- for the main logic, there are generally 4 steps to follow;
    1. iterate through all dataframe of different tech (and sub-tech)
    2. iterate through different indexes (and select that subset of the dataframe)
    3. calculate each year's value as ratio of 2020's
    4. write to summary file (the 2020 baseline data and other data are supposed to be organized in different sheets in different ways)

In [ ]:
import collections
import numpy as np

baseline_year = year_list[0]
# make a dictionary-of-dictionaries-of-dictionaries
ddd = collections.defaultdict(lambda : collections.defaultdict(dict))

debug_session = []

# iterate over sheet names and dataframes
for tech, df in sheets.items():
    # get all unique indexes in the df
    indexes = list(df['Index'].unique())
    year_dict = {}

    # iterate over the indexes
    for index in indexes:
        index_df = df[(df['Index']==index)]
        baseline = pd.to_numeric(index_df[baseline_year])
        range_list = [-1, -1]
        # iterate over years
        for year in year_list[1:]:
            try:
                target_year = pd.to_numeric(index_df[year])
                min_ratio = (target_year/baseline).min()
                max_ratio = (target_year/baseline).max()

                if np.isnan(max_ratio) or np.isnan(min_ratio):
                    if (index_df[year]==0).all():
                        range_list = [0, 0]

                else:
                    # round the floats
                    min_ratio = round(min_ratio, 4)
                    max_ratio = round(max_ratio, 4)

                    if min_ratio == max_ratio:
                        range_list = min_ratio
                    else:
                        range_list = [min_ratio, max_ratio]


            except ZeroDivisionError as e:
                if (index_df[year]==0).all():
                    range_list = [0, 0]
                else:
                    debug_session.append([str(tech), str(index), str(year)])

            finally:
                try:
                    year_dict[str(year)] = range_list.copy()
                except AttributeError:
                    year_dict[str(year)] = range_list

        ddd['Baseline-'+str(index)]['Lower Bound'][tech_chinese[tech]] = round(baseline.min(), 4)
        ddd['Baseline-'+str(index)]['Upper Bound'][tech_chinese[tech]] = round(baseline.max(), 4)
        ddd['Baseline-'+str(index)]['Lower Bound (CNY)'][tech_chinese[tech]] = round(baseline.min()/0.145, 4)
        ddd['Baseline-'+str(index)]['Upper Bound (CNY)'][tech_chinese[tech]] = round(baseline.max()/0.145, 4)
        if not baseline.min() == baseline.max():
            ddd['Baseline-'+str(index)]['Range'][tech_chinese[tech]] = "["+str(ddd['Baseline-'+str(index)]['Lower Bound (CNY)'][tech_chinese[tech]])+","+str(ddd['Baseline-'+str(index)]['Upper Bound (CNY)'][tech_chinese[tech]])+"]"
        else:
            ddd['Baseline-'+str(index)]['Range'][tech_chinese[tech]] = ddd['Baseline-'+str(index)]['Upper Bound (CNY)'][tech_chinese[tech]]
        ddd[index][tech_chinese[tech]] = year_dict.copy()

In [ ]:
# iterate over the out most layer of the 3-d dictionary
with pd.ExcelWriter(summary_path, mode='w') as writer:
    for index, index_dict in ddd.items():
        index_df = pd.DataFrame(index_dict)
        # transpose the dataframe
        if not 'Baseline' in str(index).split('-'):
            index_df = index_df.T
        index_df.to_excel(writer, sheet_name=''.join(x+' 'for x in str(index).split()[:2]))